In [1]:
import os
import tensorflow as tf
import cv2
import joblib
from ops import *
import numpy as np
import time
import datetime
import matplotlib.pyplot as plt


W0730 02:42:11.554080 139751073011456 deprecation_wrapper.py:119] From /home/obin/codes/VisualNavigation/RPF/ops.py:6: The name tf.image.resize_bilinear is deprecated. Please use tf.compat.v1.image.resize_bilinear instead.



# Parameters

In [2]:
batch_size = 4
GRU_size = 512
action_dim = 4
learning_rate = 1e-4
demo_length = 30
max_follow_length = 50
memory_dim = 512
EPOCH = 500
img_size = 256
feature_dim = 512

# Dataset

In [3]:
DATA_DIR = 'preprocessed_habitat_data2'

train_list = [os.path.join(DATA_DIR,'train',x) for x in os.listdir(os.path.join(DATA_DIR,'train'))]
valid_list = [os.path.join(DATA_DIR,'valid',x) for x in os.listdir(os.path.join(DATA_DIR,'valid'))]


In [4]:
def load_data(data_path):
    try:
        noise_data = joblib.load(data_path)
        mode = 'train' if 'train' in str(data_path) else 'valid'
        ref_data_name = str(data_path).replace(mode, 'DEMON')
        ref_data_name = ref_data_name[:ref_data_name.find('.dat.gz')-1] + '0.dat.gz'
        ref_data = joblib.load(str(ref_data_name)[2:])
        #ref_data = joblib.load(str(ref_data_name))

        noise_rgb  = np.array(noise_data['rgb'],dtype=np.float32)/255.0 * 2 - 1
        ref_rgb = np.array(ref_data['rgb'],dtype=np.float32)/255.0 * 2 - 1

        noise_data['action'] = np.array(noise_data['action'],dtype=np.int8)
        ref_data['action'] = np.array(ref_data['action'],dtype=np.int8)

        a = np.where(noise_data['action']>5)
        b = np.where(ref_data['action']>5)
        noise_data['action'][a] = noise_data['action'][a] - 5
        ref_data['action'][b] = ref_data['action'][b] - 5

        noise_action = np.eye(action_dim)[noise_data['action']]
        ref_action = np.eye(action_dim)[ref_data['action']]

        add_t = max_follow_length - len(noise_rgb)
        if add_t < 0 : 
            noise_rgb = noise_rgb[:max_follow_length]
            noise_action = noise_action[:max_follow_length]
            action_mask = np.ones_like(noise_action)
            action_mask = action_mask[:max_follow_length]
            print('its weird..', data_path)

        else:
            img_size = noise_rgb[0].shape[1]
            action_mask = np.ones_like(noise_action)

            noise_rgb = np.concatenate([noise_rgb, np.zeros([add_t, img_size, img_size, 3])],0)
            noise_action = np.concatenate([noise_action, np.zeros([add_t, action_dim])],0)
            action_mask = np.concatenate([action_mask, np.zeros([add_t, action_dim])],0)

        
    except:
        ref_rgb = np.zeros([30,256,256,3])
        ref_action = np.zeros([30,4])
        noise_rgb = np.zeros([50,256,256,3])
        noise_action = np.zeros([50,4])
        action_mask = np.zeros([50,4])
        
    result = [ref_rgb, ref_action, noise_rgb, noise_action, action_mask]
    return [np.array(x,dtype=np.float32) for x in result]

In [6]:

train_num = len(train_list)
valid_num = len(valid_list)
print('train_num {}, valid_num {}'.format(train_num, valid_num))
num_batches = int(train_num/batch_size)
print('num_batches {}'.format(num_batches))
valid_num_batches = int(valid_num/batch_size)
print('valid_num_batches {}'.format(valid_num_batches))

train_list = train_list[:batch_size*train_num]
valid_list = valid_list[:batch_size*valid_num]

tf.reset_default_graph()

    

train_dataset = tf.data.Dataset.from_tensor_slices(train_list)
dataset = train_dataset.map(lambda data_path: tf.py_func(load_data,[data_path],[tf.float32,]*5), num_parallel_calls=5)
dataset = dataset.batch(batch_size).repeat()#.shuffle(100).batch(4).repeat()

handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, dataset.output_types, dataset.output_shapes)
batch = iterator.get_next()

valid_dataset = tf.data.Dataset.from_tensor_slices(valid_list)
valid_dataset = valid_dataset.map(lambda data_path: tf.py_func(load_data,[data_path],[tf.float32]*5), num_parallel_calls=5)
valid_dataset = valid_dataset.batch(batch_size).repeat()#.shuffle(100).batch(4).repeat()

training_iterator = dataset.make_initializable_iterator()
validation_iterator = valid_dataset.make_initializable_iterator()

W0730 02:42:11.800359 139751073011456 deprecation.py:323] From <ipython-input-6-fbee81630fa6>:17: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
W0730 02:42:11.819040 139751073011456 deprecation.py:323] From <ipython-input-6-fbee81630fa6>:21: DatasetV1.output_types (from

train_num 15557, valid_num 2960
num_batches 3889
valid_num_batches 740


# Build Network

In [7]:
from model import *
rpf = RPF()
rpf.build_network_for_train(batch)
train_loss_summary = tf.summary.scalar('train_loss',rpf.loss)
valid_loss_summary = tf.summary.scalar('valid_loss',rpf.loss)

W0730 02:42:11.859769 139751073011456 deprecation_wrapper.py:119] From /home/obin/codes/VisualNavigation/RPF/model.py:23: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0730 02:42:11.869218 139751073011456 deprecation_wrapper.py:119] From /home/obin/codes/VisualNavigation/RPF/model.py:49: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0730 02:42:11.870297 139751073011456 deprecation_wrapper.py:119] From /home/obin/codes/VisualNavigation/RPF/model.py:55: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0730 02:42:11.884019 139751073011456 deprecation.py:323] From /home/obin/codes/VisualNavigation/RPF/ops.py:46: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0730 02:42:12.061755 139751073011456 deprecation.py:323] From /home/obin/codes/Visual

In [8]:
init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
network_saver = tf.train.Saver(tf.trainable_variables(),max_to_keep=100)
#saver_a = tf.train.Saver([v for v in tf.all_variables() if v.name == "a1:0"])

version_path = 'v3'
if not os.path.exists('models/'+version_path) : os.makedirs('models/'+version_path)
if not os.path.exists('logs/'+version_path) : os.makedirs('logs/'+version_path)

writer = tf.summary.FileWriter('logs/'+version_path)
#if not os.path.exists('images/'+version_path) : os.makedirs('images/'+version_path)
    
sess = tf.Session(config=config)
sess.run(init)
pre_ckpt = tf.train.get_checkpoint_state('models/'+version_path)
if pre_ckpt and pre_ckpt.model_checkpoint_path :
    past_ckpt = pre_ckpt.model_checkpoint_path
    #past_ckpt = 'models/v1ep0058step761985'
    network_saver.restore(sess,past_ckpt)
    print("1. Successfully loaded:", past_ckpt)
    ep = int(past_ckpt[past_ckpt.find('ep')+2:past_ckpt.find('ep')+6])
    step = int(past_ckpt[past_ckpt.find('step')+4:past_ckpt.find('step')+8])
else:
    print("1! Could not find old network weights")
    ep = 0
    step = 0

training_handle = sess.run(training_iterator.string_handle())
validation_handle = sess.run(validation_iterator.string_handle())

1! Could not find old network weights


In [9]:
print_step =10
save_ep = 1
#sess.graph.finalize()


# Training

In [10]:
for ep in range(ep, EPOCH):
    sess.run(training_iterator.initializer)
    
    loss_record = 0
    times = [0]
    for batch_num in range(num_batches):
        start = time.time()

        summary, _, curr_loss = sess.run([train_loss_summary, rpf.opti_step, rpf.loss],feed_dict={handle: training_handle})
        loss_record += curr_loss
        #input_feats = sess.run(rpf.record['inp_feats_t'],feed_dict={handle: training_handle})
        #print(np.min(input_feats), np.max(input_feats))
        writer.add_summary(summary,global_step=step)
        if step % print_step == 0:
            print('\t step %d, mean loss %.7f, mean step time %.3f'%(step,loss_record/(batch_num+1),np.mean(times)))
        step += 1
        times.append(time.time()-start)
        #if batch_num > 20 : break
    valid_loss = 0
    sess.run(validation_iterator.initializer)
    for v in range(valid_num_batches):
        summary, curr_valid_loss = sess.run([valid_loss_summary, rpf.loss],feed_dict={handle: validation_handle})
        writer.add_summary(summary,global_step=ep*valid_num_batches+v)
        valid_loss += curr_valid_loss
    #if v > 20 : break
    print("EPOCH %d step %d train_loss %.7f, valid_loss %.7f current_time "%(ep, step, loss_record/(batch_num+1), valid_loss/valid_num_batches))
    if ep % save_ep == 0:
        model_name = os.path.join('models',version_path,'ep%04dstep%06d'%(ep,step))
        network_saver.save(sess,model_name)
    


	 step 0, mean loss 0.7864096, mean step time 0.000
	 step 10, mean loss 0.5918349, mean step time 5.322
	 step 20, mean loss 0.5499669, mean step time 3.046
	 step 30, mean loss 0.5209820, mean step time 2.240
	 step 40, mean loss 0.5059985, mean step time 1.826
	 step 50, mean loss 0.4900840, mean step time 1.575
	 step 60, mean loss 0.4791487, mean step time 1.406
	 step 70, mean loss 0.4673374, mean step time 1.285
	 step 80, mean loss 0.4674290, mean step time 1.194
	 step 90, mean loss 0.4625746, mean step time 1.123
	 step 100, mean loss 0.4634883, mean step time 1.066
	 step 110, mean loss 0.4584613, mean step time 1.020
	 step 120, mean loss 0.4543068, mean step time 0.981
	 step 130, mean loss 0.4488501, mean step time 0.948
	 step 140, mean loss 0.4438938, mean step time 0.919
	 step 150, mean loss 0.4417311, mean step time 0.895
	 step 160, mean loss 0.4378618, mean step time 0.874
	 step 170, mean loss 0.4353606, mean step time 0.855
	 step 180, mean loss 0.4307607, mean s

	 step 1460, mean loss 0.3303921, mean step time 0.589
	 step 1470, mean loss 0.3301293, mean step time 0.589
	 step 1480, mean loss 0.3297507, mean step time 0.589
	 step 1490, mean loss 0.3295367, mean step time 0.589
	 step 1500, mean loss 0.3289325, mean step time 0.588
	 step 1510, mean loss 0.3285799, mean step time 0.588
	 step 1520, mean loss 0.3283784, mean step time 0.588
	 step 1530, mean loss 0.3281032, mean step time 0.588
its weird.. b'preprocessed_habitat_data2/train/YmJkqBEsHnH_110_num8_1.dat.gz'
	 step 1540, mean loss 0.3286392, mean step time 0.588
	 step 1550, mean loss 0.3284935, mean step time 0.587
	 step 1560, mean loss 0.3280909, mean step time 0.587
	 step 1570, mean loss 0.3277768, mean step time 0.587
	 step 1580, mean loss 0.3274396, mean step time 0.587
	 step 1590, mean loss 0.3272135, mean step time 0.587
	 step 1600, mean loss 0.3270213, mean step time 0.586
	 step 1610, mean loss 0.3267995, mean step time 0.586
	 step 1620, mean loss 0.3263426, mean ste

	 step 2910, mean loss 0.2987732, mean step time 0.572
	 step 2920, mean loss 0.2985282, mean step time 0.572
	 step 2930, mean loss 0.2983665, mean step time 0.572
	 step 2940, mean loss 0.2982571, mean step time 0.572
	 step 2950, mean loss 0.2979731, mean step time 0.572
	 step 2960, mean loss 0.2978000, mean step time 0.572
	 step 2970, mean loss 0.2977396, mean step time 0.572
	 step 2980, mean loss 0.2975862, mean step time 0.572
	 step 2990, mean loss 0.2973214, mean step time 0.572
	 step 3000, mean loss 0.2971203, mean step time 0.572
	 step 3010, mean loss 0.2970127, mean step time 0.572
	 step 3020, mean loss 0.2968137, mean step time 0.572
	 step 3030, mean loss 0.2966655, mean step time 0.572
	 step 3040, mean loss 0.2964175, mean step time 0.572
	 step 3050, mean loss 0.2962590, mean step time 0.572
	 step 3060, mean loss 0.2960806, mean step time 0.572
	 step 3070, mean loss 0.2958959, mean step time 0.571
	 step 3080, mean loss 0.2957098, mean step time 0.571
	 step 309

	 step 4290, mean loss 0.2319127, mean step time 0.553
	 step 4300, mean loss 0.2325419, mean step time 0.553
	 step 4310, mean loss 0.2324081, mean step time 0.553
	 step 4320, mean loss 0.2327071, mean step time 0.553
its weird.. b'preprocessed_habitat_data2/train/1LXtFkjw3qL_038_num3_1.dat.gz'
	 step 4330, mean loss 0.2319941, mean step time 0.553
	 step 4340, mean loss 0.2324533, mean step time 0.553
	 step 4350, mean loss 0.2324503, mean step time 0.553
	 step 4360, mean loss 0.2325789, mean step time 0.553
	 step 4370, mean loss 0.2326101, mean step time 0.553
	 step 4380, mean loss 0.2321017, mean step time 0.553
	 step 4390, mean loss 0.2317771, mean step time 0.553
	 step 4400, mean loss 0.2315404, mean step time 0.553
	 step 4410, mean loss 0.2316840, mean step time 0.553
	 step 4420, mean loss 0.2314444, mean step time 0.553
	 step 4430, mean loss 0.2311333, mean step time 0.553
	 step 4440, mean loss 0.2304997, mean step time 0.553
	 step 4450, mean loss 0.2304216, mean ste

	 step 5730, mean loss 0.2204091, mean step time 0.554
	 step 5740, mean loss 0.2202935, mean step time 0.554
	 step 5750, mean loss 0.2203646, mean step time 0.554
	 step 5760, mean loss 0.2202524, mean step time 0.554
	 step 5770, mean loss 0.2200484, mean step time 0.554
	 step 5780, mean loss 0.2197878, mean step time 0.554
	 step 5790, mean loss 0.2196946, mean step time 0.554
	 step 5800, mean loss 0.2195909, mean step time 0.554
	 step 5810, mean loss 0.2194750, mean step time 0.554
	 step 5820, mean loss 0.2194663, mean step time 0.554
	 step 5830, mean loss 0.2194643, mean step time 0.555
	 step 5840, mean loss 0.2194295, mean step time 0.555
	 step 5850, mean loss 0.2194143, mean step time 0.555
	 step 5860, mean loss 0.2193306, mean step time 0.555
	 step 5870, mean loss 0.2191473, mean step time 0.555
	 step 5880, mean loss 0.2190262, mean step time 0.555
	 step 5890, mean loss 0.2191389, mean step time 0.555
	 step 5900, mean loss 0.2190391, mean step time 0.555
	 step 591

...

	 step 31080, mean loss 0.0388064, mean step time 0.556
	 step 31090, mean loss 0.0388328, mean step time 0.556
its weird.. b'preprocessed_habitat_data2/train/Pm6F8kyY3z2_257_num8_2.dat.gz'
	 step 31100, mean loss 0.0388201, mean step time 0.556
	 step 31110, mean loss 0.0388122, mean step time 0.556
EPOCH 7 step 31112 train_loss 0.0388103, valid_loss 0.4904584 current_time 
	 step 31120, mean loss 0.0406144, mean step time 0.540
	 step 31130, mean loss 0.0434931, mean step time 0.545
	 step 31140, mean loss 0.0400758, mean step time 0.547
	 step 31150, mean loss 0.0366080, mean step time 0.548
	 step 31160, mean loss 0.0367911, mean step time 0.548
	 step 31170, mean loss 0.0373963, mean step time 0.549
	 step 31180, mean loss 0.0360000, mean step time 0.549
	 step 31190, mean loss 0.0365796, mean step time 0.549
	 step 31200, mean loss 0.0354147, mean step time 0.550
	 step 31210, mean loss 0.0351692, mean step time 0.550
	 step 31220, mean loss 0.0341848, mean step time 0.550
	 step

	 step 32460, mean loss 0.0354423, mean step time 0.555
	 step 32470, mean loss 0.0353959, mean step time 0.555
	 step 32480, mean loss 0.0353693, mean step time 0.555
	 step 32490, mean loss 0.0354046, mean step time 0.555
	 step 32500, mean loss 0.0354810, mean step time 0.555
	 step 32510, mean loss 0.0354266, mean step time 0.555
	 step 32520, mean loss 0.0354387, mean step time 0.555
	 step 32530, mean loss 0.0353824, mean step time 0.555
	 step 32540, mean loss 0.0353713, mean step time 0.555
	 step 32550, mean loss 0.0353665, mean step time 0.555
	 step 32560, mean loss 0.0353299, mean step time 0.555
	 step 32570, mean loss 0.0352815, mean step time 0.555
	 step 32580, mean loss 0.0352803, mean step time 0.555
	 step 32590, mean loss 0.0352762, mean step time 0.555
	 step 32600, mean loss 0.0352494, mean step time 0.555
	 step 32610, mean loss 0.0351808, mean step time 0.555
	 step 32620, mean loss 0.0351577, mean step time 0.555
	 step 32630, mean loss 0.0351453, mean step tim

	 step 33890, mean loss 0.0343165, mean step time 0.556
	 step 33900, mean loss 0.0343164, mean step time 0.556
	 step 33910, mean loss 0.0342827, mean step time 0.556
	 step 33920, mean loss 0.0343133, mean step time 0.556
	 step 33930, mean loss 0.0342838, mean step time 0.556
	 step 33940, mean loss 0.0342967, mean step time 0.556
	 step 33950, mean loss 0.0342745, mean step time 0.556
	 step 33960, mean loss 0.0342740, mean step time 0.556
	 step 33970, mean loss 0.0342772, mean step time 0.556
	 step 33980, mean loss 0.0343061, mean step time 0.556
	 step 33990, mean loss 0.0343531, mean step time 0.556
	 step 34000, mean loss 0.0343357, mean step time 0.556
	 step 34010, mean loss 0.0343422, mean step time 0.556
	 step 34020, mean loss 0.0343379, mean step time 0.556
	 step 34030, mean loss 0.0343271, mean step time 0.556
	 step 34040, mean loss 0.0343748, mean step time 0.556
	 step 34050, mean loss 0.0343805, mean step time 0.556
	 step 34060, mean loss 0.0343602, mean step tim

	 step 35260, mean loss 0.0324945, mean step time 0.553
	 step 35270, mean loss 0.0323436, mean step time 0.553
	 step 35280, mean loss 0.0323180, mean step time 0.553
	 step 35290, mean loss 0.0322054, mean step time 0.553
	 step 35300, mean loss 0.0322306, mean step time 0.553
	 step 35310, mean loss 0.0321412, mean step time 0.553
	 step 35320, mean loss 0.0317765, mean step time 0.553
	 step 35330, mean loss 0.0316389, mean step time 0.553
its weird.. b'preprocessed_habitat_data2/train/29hnd4uzFmX_114_num3_2.dat.gz'
	 step 35340, mean loss 0.0315290, mean step time 0.553
	 step 35350, mean loss 0.0315597, mean step time 0.553
	 step 35360, mean loss 0.0316333, mean step time 0.553
	 step 35370, mean loss 0.0314819, mean step time 0.553
	 step 35380, mean loss 0.0311249, mean step time 0.553
	 step 35390, mean loss 0.0308918, mean step time 0.553
	 step 35400, mean loss 0.0309076, mean step time 0.553
	 step 35410, mean loss 0.0308283, mean step time 0.553
	 step 35420, mean loss 0.

KeyboardInterrupt: 

In [ ]:
b = sess.run(batch,feed_dict={handle: training_handle})

In [ ]:
b[0].shape

In [ ]:
sess.run(validation_iterator.initializer)

In [ ]:
valid_loss = 0
for i in range(1000):
    curr_valid_loss = sess.run(loss,feed_dict={handle: validation_handle})
    valid_loss += curr_valid_loss
    print(curr_valid_loss)
    
print(valid_loss/1000)

# Run Example

In [ ]:
sess.run(training_iterator.initializer)

In [ ]:
print(rec.keys())

In [ ]:
def pprint(arr):
    log = ''
    for k in arr:
        log += '%.04f '%(k)
    print(log)
    return

In [ ]:
for i in range(len(dat[2])):
    print('eta t+1 - eta t = %.04f - %.04f = %.04f'%(rec['eta'][i-1],rec['eta'][i], rec['eta'][i]-rec['eta'][i-1] -1))
    

In [ ]:
rec, dat, losss = sess.run([record, batch, loss],feed_dict={handle: validation_handle})
for i in range(len(dat[2])):
    view_img = np.zeros([256,256*2,3])
    plt.subplot(121)
    if int(rec['eta'][i]) > 29 : 
        plt.imshow((dat[0][29]+1)/2.)
        view_img[:,:256,:] = (dat[0][29]+1)/2.
    elif i >= 1: 
        plt.imshow((dat[0][int(rec['eta'][i-1])]+1)/2.)
        view_img[:,:256,:] = (dat[0][int(rec['eta'][i-1])]+1)/2.

    plt.subplot(122)
    plt.imshow((dat[2][i]+1)/2.)
    view_img[:,256:,:] = (dat[2][i]+1)/2.
    plt.show()
    plt.imsave('img_{}.png'.format(i),view_img)
    act = rec['actions'][i].squeeze()
    dem_act = dat[3][i]
    if np.argmax(act) != np.argmax(dem_act) : print('wrong!')
    print('eta %.04f actions %.04f %.04f %.04f %.04f '%(rec['eta'][i-1], act[0],act[1],act[2],act[3]) )
    print('eta %.04f actions %.04f %.04f %.04f %.04f '%(rec['eta'][i-1], dem_act[0],dem_act[1],dem_act[2],dem_act[3]) )
    pprint(rec['attention'][i])
    plt.plot(range(30),rec['attention'][i])
    plt.savefig('attention_{}.png'.format(i))
    plt.show()
    
    